In [1]:
import cfg

running price stream
running transaction stream
running account update polling


In [2]:
cfg.account

In [3]:
cfg.account.trades

In [4]:
type(cfg.account.trades)

list

In [5]:
for t in cfg.account.trades:
    print(t)
    break

id: '170994'
instrument: EUR_USD
price: 1.13014
openTime: '2022-01-07T04:42:39.020810339Z'
state: OPEN
initialUnits: -381.0
initialMarginRequired: 8.6123
currentUnits: -381.0
realizedPL: 0.0
unrealizedPL: -0.0724
marginUsed: 8.6126
financing: 0.0
takeProfitOrderID: '170995'
stopLossOrderID: '170996'
trailingStopLossOrderID: '170997'


In [6]:
def check_breakeven_for_instrument(instrument: str, trades: list):
    all_breakeven = []
    for t in trades:
        if t.instrument == instrument:
            for o in cfg.account.orders:
                if o.id == t.stopLossOrderID:
                    all_breakeven.append(
                    (t.currentUnits > 0 ) and (o.price >=t.price)
                    or
                    (t.currentUnits <0 and o.price<=t.price))
    return all(all_breakeven)

In [7]:
check_breakeven_for_instrument('AUD_USD', cfg.account.trades)

True

In [8]:
for o in cfg.account.orders:
    print(o)
    break

id: '170637'
createTime: '2022-01-06T15:39:21.373806140Z'
state: PENDING
type: TAKE_PROFIT
tradeID: '170636'
price: 0.83202
timeInForce: GTC
triggerCondition: DEFAULT


In [9]:
cfg.account.balance

1565.1642

In [10]:
cfg.instruments['EUR_USD']

{'name': 'EUR_USD',
 'type': 'CURRENCY',
 'displayName': 'EUR/USD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'minimumTradeSize': '1.0',
 'maximumTrailingStopDistance': '1.0',
 'minimumTrailingStopDistance': '0.0005',
 'maximumPositionSize': '0.0',
 'maximumOrderUnits': '100000000.0',
 'marginRate': '0.02',
 'bid': 1.13387,
 'ask': 1.13401,
 'spread': 0.00014}

In [11]:
cfg.account.marginAvailable

1513.6273